# MLP로 Iris 분류하기
by uramoon@kw.ac.kr
(<a href="https://raw.githubusercontent.com/ronreiter/interactive-tutorials/master/LICENSE">Apache 2.0 License</a>)

## 데이터셋 만들기
seaborn에서 제공하는 Iris 데이터셋을 사용합니다.<br>
seaborn은 데이터 시각화 도구입니다. (https://seaborn.pydata.org/)

In [1]:
# seaborn 라이브러리로 iris 데이터셋 DataFrame으로 읽어오기
import seaborn as sns
iris = sns.load_dataset("iris")

### 데이터셋 살펴보기

In [2]:
# TODO: 무작위로 10개의 샘플을 확인해보세요.
iris.sample(10)

,sepal_length,sepal_width,petal_length,petal_width,species
77,6.7,3.0,5.0,1.7,versicolor
45,4.8,3.0,1.4,0.3,setosa
29,4.7,3.2,1.6,0.2,setosa
82,5.8,2.7,3.9,1.2,versicolor
105,7.6,3.0,6.6,2.1,virginica
43,5.0,3.5,1.6,0.6,setosa
28,5.2,3.4,1.4,0.2,setosa
122,7.7,2.8,6.7,2.0,virginica
42,4.4,3.2,1.3,0.2,setosa
87,6.3,2.3,4.4,1.3,versicolor


In [3]:
# TODO: 각 컬럼의 데이터 타입과 null 값을 지닌 컬럼이 있는지 확인해보세요.
# Hint: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html
iris.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


꽃받침(sepal)의 길이, 너비와 꽃잎(petal)의 길이, 너비로 어떤 꽃인지 예측할 것입니다.

In [4]:
# TODO: 어떤 꽃들이 있는지 알아봅시다.
# Hint: https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html
iris.value_counts()

sepal_length  sepal_width  petal_length  petal_width  species   
5.8           2.7          5.1           1.9          virginica     2
6.2           2.2          4.5           1.5          versicolor    1
              2.9          4.3           1.3          versicolor    1
              3.4          5.4           2.3          virginica     1
6.3           2.3          4.4           1.3          versicolor    1
                                                                   ..
5.4           3.9          1.3           0.4          setosa        1
                           1.7           0.4          setosa        1
5.5           2.3          4.0           1.3          versicolor    1
              2.4          3.7           1.0          versicolor    1
7.9           3.8          6.4           2.0          virginica     1
Length: 149, dtype: int64

### 훈련 데이터, 검증 데이터, 테스트 데이터로 나누기
1. 훈련 데이터: 모델을 훈련할 때 직접적으로 사용
2. 검증 데이터: 모델을 훈련할 때 간접적으로 사용
3. 테스트 데이터: 훈련이 끝난 모델의 성능을 평가할 때 사용

일반적으로 훈련 (60%), 검증 (20%), 테스트 (20%)로 나눕니다. <br>
훈련/검증 (80%), 테스트 (20%)로 나눈 후 훈련/검증을 다시 훈련 (80%)과 검증 (20%)으로 나누기도 합니다.<br>

모두 같은 결과를 얻기 위해 train_test_split 함수에 random_state 42를 사용합니다.<br>
42는 '은하수를 여행하는 히치하이커를 위한 안내서'에서 나온 숫자입니다.

In [5]:
from sklearn.model_selection import train_test_split

# 훈련/검증 (80%), 테스트 (20%)
train, test = train_test_split(iris, test_size=0.2, random_state=42)

In [6]:
# TODO: train과 test 샘플 수를 확인하세요.
print(len(train))
print(len(test))

120
30


## X값과 y값 나누기
지도 학습에서는 데이터 (X)와 답 (y)을 분리하여 모델에 넣어줍니다.

In [7]:
train_X = train.drop(['species'], axis=1) # species 열만 제거
train_y = train['species'] # species 열만 가져오기

# TODO
test_X = test.drop(['species'], axis=1)
test_y = test['species'] 

### y값 숫자로 변환하기

In [8]:
# 현재의 y값
train_y

22         setosa
15         setosa
65     versicolor
11         setosa
42         setosa
          ...    
71     versicolor
106     virginica
14         setosa
92     versicolor
102     virginica
Name: species, Length: 120, dtype: object

In [9]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_y)

LabelEncoder()

In [10]:
# 다음의 클래스들이 0, 1, 2로 맵핑됩니다.
le.classes_

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [11]:
# 레이블 숫자로 변환하기
train_y = le.transform(train_y)
train_y

array([0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2, 1, 2, 1, 2,
       1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2,
       1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 0, 0, 1, 1, 2, 1, 2, 2, 1,
       0, 0, 2, 2, 0, 0, 0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2, 1, 2,
       1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2, 1, 2,
       1, 1, 2, 2, 0, 1, 2, 0, 1, 2])

In [12]:
# TODO
test_y = le.transform(test_y)
test_y

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

## 전처리 하기
Min-max normalization으로 모든 인자의 값을 0과 1사이의 값으로 변환합니다. <br>(https://en.wikipedia.org/wiki/Feature_scaling#Rescaling_(min-max_normalization))<br>
**(중요) 전처리 기법도 훈련 데이터만 사용하여 학습한 후 다른 데이터에 적용합니다.**

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# train_X의 min, max값으로 scaler를 설정합니다.
scaler.fit(train_X)

MinMaxScaler()

In [14]:
# 0부터 1사이의 NumPy 배열로 변환
train_X = scaler.transform(train_X)

# TODO
test_X = scaler.transform(test_X)

In [15]:
# 잘 변환됐는지 확인
import numpy as np

print(np.min(train_X), np.max(train_X))
print(np.min(test_X), np.max(test_X))

0.0 1.0
0.0 1.0588235294117647


## 모델 만들기
MLP에서 숨겨진 층의 노드 개수는 입력층의 2/3 정도로 설정하는 것이 일반적이지만 최적이 아닐 수도 있으니 직접 실험을 해가며 설정하는 것이 좋습니다. 출력층의 활성화 함수는 출력 중 큰 값에 높은 확률을 할당해주는 함수를 사용합니다. (https://en.wikipedia.org/wiki/Softmax_function)

In [16]:
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping

model = models.Sequential()
model.add(layers.Input(shape=(train_X.shape[1],))) # 입력층의 모양
model.add(layers.Dense(4, activation='relu')) # 숨겨진 층
model.add(layers.Dense(3, activation='softmax')) # 출력층의 모양 및 활성화 함수, 

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 20        
                                                                 
 dense_1 (Dense)             (None, 3)                 15        
                                                                 
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


In [17]:
# TODO: 컴파일하기
# 레이블에 one-hot 인코딩을 사용하지 않았으므로 loss에 sparse_categorical_crossentropy를 사용합니다.

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## 모델 훈련하기
훈련 데이터를 사용하여 인공신경망의 가중치를 조절합니다. 한 epoch이 끝나면 (훈련 데이터를 완전히 한 번 보고나면) 검증 데이터의 loss를 측정합니다. 훈련 데이터의 loss는 줄어드는데 검증 데이터의 loss가 증가하면 과적합이 일어나는 신호이므로 훈련을 중단합니다.

In [18]:
# TODO: 훈련 데이터와 정답으로 모델을 훈련합니다.

model.fit(train_X, train_y, epochs=10000, 
            callbacks=EarlyStopping(),  # val_loss를 관찰하여 과적합이 일어나면 훈련을 중단합니다.
            validation_split=0.2) # 훈련 데이터의 20%를 검증 데이터로 사용합니다.

Epoch 1/10000
3/3 [==============================] - 1s 233ms/step - loss: 1.1333 - accuracy: 0.3646 - val_loss: 1.1988 - val_accuracy: 0.2083
Epoch 2/10000
3/3 [==============================] - 0s 26ms/step - loss: 1.1291 - accuracy: 0.3646 - val_loss: 1.1928 - val_accuracy: 0.2083


일시적인 현상에 콜백 함수가 속을 수 있으니 val_loss가 감소하지 않아도 10번의 epochs을 참을 수 있도록 설정해봅시다.

In [19]:
model.fit(train_X, train_y, epochs=10000, 
            callbacks=EarlyStopping(patience=10),  # val_loss를 관찰하여 과적합이 일어나면 훈련을 중단합니다.
            validation_split=0.2) # 훈련 데이터의 20%를 검증 데이터로 사용합니다.

Epoch 1/10000
3/3 [==============================] - 0s 85ms/step - loss: 1.1253 - accuracy: 0.3646 - val_loss: 1.1870 - val_accuracy: 0.2083
Epoch 2/10000
3/3 [==============================] - 0s 20ms/step - loss: 1.1213 - accuracy: 0.3646 - val_loss: 1.1817 - val_accuracy: 0.2083
Epoch 3/10000
3/3 [==============================] - 0s 32ms/step - loss: 1.1175 - accuracy: 0.3646 - val_loss: 1.1768 - val_accuracy: 0.2083
Epoch 4/10000
3/3 [==============================] - 0s 20ms/step - loss: 1.1139 - accuracy: 0.3542 - val_loss: 1.1718 - val_accuracy: 0.2083
Epoch 5/10000
3/3 [==============================] - 0s 33ms/step - loss: 1.1102 - accuracy: 0.3542 - val_loss: 1.1670 - val_accuracy: 0.2083
Epoch 6/10000
3/3 [==============================] - 0s 27ms/step - loss: 1.1067 - accuracy: 0.3542 - val_loss: 1.1625 - val_accuracy: 0.2083
Epoch 7/10000
3/3 [==============================] - 0s 22ms/step - loss: 1.1033 - accuracy: 0.3542 - val_loss: 1.1582 - val_accuracy: 0.2083
Epoch 

## 테스트 데이터로 모델 평가하기

In [20]:
# TODO: 테스트 데이터와 정답으로 모델을 평가합니다.
# Hint: https://keras.io/api/models/model_training_apis/#evaluate-method
model.evaluate(test_X, test_y)

1/1 [==============================] - 0s 20ms/step - loss: 0.1032 - accuracy: 0.9667


[0.10315947234630585, 0.9666666388511658]

## 새로운 모델 만들어보기
테스트 데이터에서 96.7%보다 높은 정확도를 달성할 수 있나요?

In [32]:
# TODO: 자유롭게 실험, 너무 많은 시간을 쓰지 않습니다.
model = models.Sequential()
model.add(layers.Input(shape=(train_X.shape[1],))) # 입력층의 모양
model.add(layers.BatchNormalization()) # 숨겨진 층
model.add(layers.Dense(4, activation='relu')) # 숨겨진 층
model.add(layers.Dense(3, activation='softmax')) # 출력층의 모양 및 활성화 함수, 

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_X, train_y, epochs=10000, 
            callbacks=EarlyStopping(patience=10),  # val_loss를 관찰하여 과적합이 일어나면 훈련을 중단합니다.
            validation_split=0.2) # 훈련 데이터의 20%를 검증 데이터로 사용합니다.

Epoch 1/10000
3/3 [==============================] - 1s 85ms/step - loss: 1.0516 - accuracy: 0.4583 - val_loss: 0.9872 - val_accuracy: 0.7083
Epoch 2/10000
3/3 [==============================] - 0s 17ms/step - loss: 1.0397 - accuracy: 0.5104 - val_loss: 0.9814 - val_accuracy: 0.7083
Epoch 3/10000
3/3 [==============================] - 0s 17ms/step - loss: 1.0474 - accuracy: 0.5104 - val_loss: 0.9758 - val_accuracy: 0.7083
Epoch 4/10000
3/3 [==============================] - 0s 25ms/step - loss: 1.0233 - accuracy: 0.5833 - val_loss: 0.9704 - val_accuracy: 0.7083
Epoch 5/10000
3/3 [==============================] - 0s 15ms/step - loss: 1.0014 - accuracy: 0.5417 - val_loss: 0.9653 - val_accuracy: 0.7083
Epoch 6/10000
3/3 [==============================] - 0s 19ms/step - loss: 1.0145 - accuracy: 0.5833 - val_loss: 0.9603 - val_accuracy: 0.7083
Epoch 7/10000
3/3 [==============================] - 0s 15ms/step - loss: 0.9891 - accuracy: 0.5938 - val_loss: 0.9555 - val_accuracy: 0.7083
Epoch 

In [33]:
model.evaluate(test_X, test_y)

1/1 [==============================] - 0s 24ms/step - loss: 0.0945 - accuracy: 0.9667


[0.09448742121458054, 0.9666666388511658]